# **미니프로젝트 4차 1대1 문의 내용 유형 분류기**
# 단계3 : Text classification

### 문제 정의
> 1:1 문의 내용 분류 문제<br>
> 1. 문의 내용 분석
> 2. 문의 내용 분류 모델 성능 평가
### 학습 데이터
> * 1:1 문의 내용 데이터 : train.csv

### 변수 소개
> * text : 문의 내용
> * label : 문의 유형

### References
> * Machine Learning
>> * [sklearn-tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)
> * Deep Learning
>> * [Google Tutorial](https://developers.google.com/machine-learning/guides/text-classification)
>> * [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)
>> * [Keras-tutorial](https://keras.io/examples/nlp/text_classification_from_scratch/)
>> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)

## 1. 개발 환경 설정

### 1-1. 라이브러리 설치

In [3]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.6/575.6 KB 50.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 17.6 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=3a76638c6a75ed6a00bc36344b743fe86ffcfee7117797c830f1866c95fd8c61
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


### 1-2. 라이브러리 import

In [4]:
from mecab import MeCab
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wget,os
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
import wget,os

### 1-3. 한글 글꼴 설정(Windows)

In [3]:
# if not os.path.exists("malgun.ttf"): 
#     wget.download("https://www.wfonts.com/download/data/2016/06/13/malgun-gothic/malgun.ttf")
# if 'malgun' not in fm.fontManager.findfont("Malgun Gothic"):
#     fm.fontManager.addfont("malgun.ttf")
# if plt.rcParams['font.family']!= ["Malgun Gothic"]:
#     plt.rcParams['font.family']= [font for font in fm.fontManager.ttflist if 'malgun.ttf' in font.fname][-1].name
# plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결
# assert plt.rcParams['font.family'] == ["Malgun Gothic"], "한글 폰트가 설정되지 않았습니다."
# FONT_PATH = "malgun.ttf"

In [5]:
!sudo apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 0s (56.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 122349 files and dir

### 1-4. 자바 경로 설정(Windows)

In [5]:
# os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-19"

### 1-3. 한글 글꼴 설정(Colab)

In [6]:
!sudo apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [7]:
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=FONT_PATH, size=10).get_name()
fm.fontManager.addfont(FONT_PATH)
print(font_name)

plt.rcParams['font.family']=[font_name]
plt.rcParams['axes.unicode_minus']=False
assert plt.rcParams['font.family'] == [font_name], "한글 폰트가 설정되지 않았습니다."

NanumGothic


### 1-4. 구글드라이브 연결(Colab)

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. 전처리한 데이터 불러오기
* 1, 2일차에 전처리한 데이터를 불러옵니다.
* sparse data에 대해서는 scipy.sparse.load_npz 활용

In [9]:
import scipy.sparse
import numpy as np

# sparse matrix 불러오기
X_tfidf_train = scipy.sparse.load_npz('/content/drive/MyDrive/aivle/4차 미니프로젝트/X_tfidf_train.npz')
X_tfidf_val = scipy.sparse.load_npz('/content/drive/MyDrive/aivle/4차 미니프로젝트/X_tfidf_val.npz')
X_tfidf_te = scipy.sparse.load_npz('/content/drive/MyDrive/aivle/4차 미니프로젝트/X_tfidf_te.npz')

# numpy array 불러오기
X_mor_sequence_train = np.load('/content/drive/MyDrive/aivle/4차 미니프로젝트/X_mor_sequence_train.npy')
X_mor_sequence_val = np.load('/content/drive/MyDrive/aivle/4차 미니프로젝트/X_mor_sequence_val.npy')
X_mor_sequence_te = np.load('/content/drive/MyDrive/aivle/4차 미니프로젝트/X_mor_sequence_te.npy')

# y 불러오기
y_train = np.load('/content/drive/MyDrive/aivle/4차 미니프로젝트/y_train.npy')
y_val = np.load('/content/drive/MyDrive/aivle/4차 미니프로젝트/y_val.npy')

In [10]:
x_test = scipy.sparse.load_npz('/content/drive/MyDrive/aivle/4차 미니프로젝트/x_test.npz')

In [43]:
x_test_sp_tfidf = scipy.sparse.load_npz('/content/drive/MyDrive/aivle/4차 미니프로젝트/x_test_sp_tfidf.npz')

## 3. Machine Learning(N-grams)
* N-gram으로 전처리한 데이터를 이용하여 3개 이상의 Machine Learning 모델 학습 및 성능 분석
> * [sklearn-tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

### 3-1. Model 1

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report

In [12]:
X_tfidf_train.shape, x_test.shape, y_train.shape, X_tfidf_val.shape, y_val.shape

((2084, 8568), (929, 8568), (2084,), (927, 8568), (927,))

In [18]:
prob = model_linear.predict_proba(x_test)
prob = pd.DataFrame(prob)

In [19]:
prob[:5]

,0,1,2,3,4
0,0.070780,0.142532,0.073517,0.707200,0.005971
1,0.228574,0.140533,0.223782,0.397791,0.009319
2,0.966506,0.017278,0.006340,0.006346,0.003530
3,0.962792,0.007515,0.023959,0.004239,0.001496
4,0.630193,0.237220,0.081147,0.029603,0.021838


In [20]:
prob.to_csv('logistic_prob.csv', index = False)

In [ ]:
model_linear = LogisticRegression(C = 5, max_iter = 1000, penalty = 'l2', solver =  'liblinear')
model_linear.fit(X_tfidf_train, y_train)
y_pred = model_linear.predict(x_test_sp_tfidf)

In [44]:
model_linear = LogisticRegression(C = 5, max_iter = 1000, penalty = 'l2', solver =  'liblinear')
model_linear.fit(X_tfidf_train, y_train)
y_pred = model_linear.predict(x_test_sp_tfidf)
print(classification_report(y_pred, y_val))
print(f1_score(y_pred, y_val, average='macro'))

ValueError: ignored

In [96]:
y_pred_df = pd.DataFrame(y_pred, columns = ['label'])
y_pred_df.reset_index(inplace=True)
y_pred_df.rename(columns={'index': 'id'}, inplace=True)
y_pred_df[:5]

,id,label
0,0,3
1,1,3
2,2,0
3,3,0
4,4,0


In [98]:
y_pred_df.to_csv('y_pred.csv', index = False)

In [ ]:
print(classification_report(y_val, y_pred))

### 3-2. Model 2

In [ ]:
model_rf = RandomForestClassifier()
model_rf.fit(X_tfidf_train, y_train)
y_pred = model_rf.predict(X_tfidf_val)
f1_score(y_val, y_pred, average = 'weighted')

0.7154423639639229

### 3-3. Model 3

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

X_mor_sequence_train_str = np.array([str(x) for x in X_mor_sequence_train])
X_mor_sequence_val_str = np.array([str(x) for x in X_mor_sequence_val])
# X_mor_sequence_te_str = np.array([str(x) for x in X_mor_sequence_te])

vectorizer = CountVectorizer(ngram_range=(1,2))
X_train_ngram = vectorizer.fit_transform(X_mor_sequence_train_str)
X_val_ngram = vectorizer.transform(X_mor_sequence_val_str)

model_linear.fit(X_train_ngram, y_train)
y_pred = model_linear.predict(X_val_ngram)
f1_score(y_val, y_pred, average = 'weighted')

0.788217162389398

In [ ]:
from sklearn.svm import SVC

model_svm = SVC()
model_svm.fit(X_tfidf_train, y_train)
y_pred = model_svm.predict(X_tfidf_val)
f1_score(y_val, y_pred, average = 'weighted')

0.7584122547886745

### 3-4. Model 4

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Conv1D, LSTM, Dropout
from tensorflow.keras.models import Sequential, Model

In [ ]:
X_tfidf_train.shape, y_val.shape, y_train.shape

((2084, 62966), (927,), (2084,))

In [ ]:
clear_session()

model = Sequential()

model.add(Input(shape = (62966)))
model.add(Flatten())
model.add(Dense(2048, activation = 'relu'))
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(512, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = keras.losses.sparse_categorical_crossentropy,
              optimizer = 'adam',
              metrics = 'accuracy')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 62966)             0         
                                                                 
 dense (Dense)               (None, 2048)              128956416 
                                                                 
 dense_1 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_2 (Dense)             (None, 512)               524800    
                                                                 
 dense_3 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 131,581,957
Trainable params: 131,581,957
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor = 'val_loss',
                   min_delta = 0,
                   patience = 5,
                   restore_best_weights = True)
model.fit(X_tfidf_train.toarray(), y_train, validation_data = (X_tfidf_val.toarray(), y_val), verbose = 1, epochs = 100, callbacks = [es])

Epoch 1/100
66/66 [==============================] - 10s 63ms/step - loss: 0.9571 - accuracy: 0.6291 - val_loss: 0.6184 - val_accuracy: 0.7875
Epoch 2/100
66/66 [==============================] - 2s 35ms/step - loss: 0.0681 - accuracy: 0.9794 - val_loss: 0.9427 - val_accuracy: 0.7961
Epoch 3/100
66/66 [==============================] - 2s 37ms/step - loss: 0.0058 - accuracy: 0.9986 - val_loss: 0.8693 - val_accuracy: 0.8101
Epoch 4/100
66/66 [==============================] - 2s 32ms/step - loss: 1.4763e-04 - accuracy: 1.0000 - val_loss: 0.9127 - val_accuracy: 0.8112
Epoch 5/100
66/66 [==============================] - 2s 32ms/step - loss: 6.1100e-05 - accuracy: 1.0000 - val_loss: 0.9423 - val_accuracy: 0.8145
Epoch 6/100
66/66 [==============================] - 3s 40ms/step - loss: 3.4713e-05 - accuracy: 1.0000 - val_loss: 0.9648 - val_accuracy: 0.8145


In [ ]:
y_pred = model.predict(X_tfidf_val.toarray())
confusion_matrix(y_val, y_pred)
classification_report(y_val, y_pred)

29/29 [==============================] - 0s 9ms/step


ValueError: ignored

### 3-5. Model 5

In [22]:
from lightgbm import LGBMClassifier

model_lgbm = LGBMClassifier(n_estimators=500, max_depth=9, min_child_weight=5, n_jobs=-1)
model_lgbm.fit(X_tfidf_train, y_train)
y_pred = model_lgbm.predict(X_tfidf_val)
f1_score(y_val, y_pred, average = 'weighted')

0.7396517856114904

In [ ]:
!pip install optuna

In [34]:
from sklearn.metrics import mean_squared_error

In [32]:
def RMSE(y, y_pred):
    rmse = mean_squared_error(y, y_pred) ** 0.5
    return rmse

In [36]:
from optuna.samplers import TPESampler
import optuna
from optuna.integration import XGBoostPruningCallback
import lightgbm as lgb

sampler = TPESampler(seed = 10)


def objective(trial):
    dtrain = lgb.Dataset(X_tfidf_train, label=y_train)
    dtest = lgb.Dataset(X_tfidf_val, label=y_val)

    param = {
        'objective': 'multiclass', # 회귀
        'verbose': -1,
        'metric': None, 
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }

    model = lgb.LGBMClassifier(**param)
    lgb_model = model.fit(X_tfidf_train, y_train, eval_set=[(X_tfidf_val, y_val)], verbose=0, early_stopping_rounds=25)
    y_pred = lgb_model.predict(X_tfidf_val)
    f1_macro = f1_score(y_val, y_pred, average='macro')
    return f1_macro

study_lgb = optuna.create_study(direction='maximize', sampler=sampler)
study_lgb.optimize(objective, n_trials=100)

[I 2023-04-07 06:09:19,652] A new study created in memory with name: no-name-48865dca-5c83-49d1-a6e1-51fca9f22b30
<ipython-input-36-642066095754>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
<ipython-input-36-642066095754>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
/usr/local/lib/python3.9/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead

KeyboardInterrupt: ignored

In [37]:
trial = study_lgb.best_trial
trial_params = trial.params
print('Best Trial: score {},\nparams {}'.format(trial.value, trial_params))

Best Trial: score 0.7576613564800573,
params {'max_depth': 3, 'learning_rate': 0.009888319202599477, 'n_estimators': 544, 'min_child_samples': 49, 'subsample': 0.9537322871331387}


### 3-6. Hyperparameter Tuning(Optional) 
* Manual Search, Grid search, Bayesian Optimization, TPE...
> * [grid search tutorial sklearn](https://scikit-learn.org/stable/modules/grid_search.html)
> * [optuna tutorial](https://optuna.org/#code_examples)
> * [ray-tune tutorial](https://docs.ray.io/en/latest/tune/examples/tune-sklearn.html)

In [14]:
!pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import GridSearchCV

In [25]:
model = LogisticRegression()

params ={
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_search = GridSearchCV(model,
                           params, 
                           cv=5,
                           scoring = 'f1')

grid_search.fit(X_tfidf_train, y_train)

best_model = grid_search.best_estimator_
best_model.fit(X_tfidf_train, y_train)

# 테스트 데이터에 대한 예측 및 평가
y_pred = best_model.predict(X_tfidf_val)
f1 = f1_score(y_val, y_pred, average = 'weighted')
print(f"F1-score: {f1}")

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/usr/local/lib/pyt

F1-score: 0.2513492536678681


In [ ]:
print(grid_search.best_score_)

0.7332042058660764


In [ ]:
# 모델 객체 생성
rfc = RandomForestClassifier()

# 목적 함수 설정
def objective_function(n_estimators, max_depth):
    # 하이퍼파라미터 값 설정
    rfc.set_params(n_estimators=int(n_estimators), max_depth=int(max_depth))

    # 모델 학습 및 예측
    rfc.fit(X_tfidf_train, y_train)
    y_pred = rfc.predict(X_tfidf_val)

    # F1-score 계산
    score = f1_score(y_val, y_pred, average='macro')

    # 목적 함수 값 반환
    return score

# 범위 설정
pbounds = {
    'n_estimators': (50, 150),
    'max_depth': (3, 7)
}

# Bayesian Optimization 수행
optimizer = BayesianOptimization(
    f=objective_function,
    pbounds=pbounds,
    random_state=42,
)

optimizer.maximize(init_points=5, n_iter=10)

# 최적의 하이퍼파라미터 값 출력
print(optimizer.max)


|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
| 1         | 0.1189    | 4.498     | 145.1     |
| 2         | 0.1189    | 5.928     | 109.9     |
| 3         | 0.1189    | 3.624     | 65.6      |
| 4         | 0.1189    | 3.232     | 136.6     |
| 5         | 0.1189    | 5.404     | 120.8     |
| 6         | 0.1212    | 6.062     | 50.0      |
| 7         | 0.1189    | 5.343     | 50.31     |
| 8         | 0.1189    | 3.405     | 102.2     |
| 9         | 0.1189    | 3.023     | 96.38     |
| 10        | 0.1189    | 5.67      | 114.2     |
| 11        | 0.1189    | 3.13      | 71.52     |
| 12        | 0.1189    | 4.767     | 132.4     |
| 13        | 0.1189    | 3.074     | 93.83     |
| 14        | 0.1189    | 4.581     | 98.74     |
| 15        | 0.1189    | 5.761     | 62.24     |
{'target': 0.12119035575788231, 'params': {'max_depth': 6.062187002063443, 'n_estimators': 50.001163475536615}}


## 4. Deep Learning(Sequence)
* Sequence로 전처리한 데이터를 이용하여 DNN, 1-D CNN, LSTM 등 3가지 이상의 deep learning 모델 학습 및 성능 분석
> * [Google Tutorial](https://developers.google.com/machine-learning/guides/text-classification)
> * [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)
> * [Keras-tutorial](https://keras.io/examples/nlp/text_classification_from_scratch/)

### 4-1. DNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Dense, Embedding, Flatten, Conv1D, LSTM, Dropout
from tensorflow.keras.models import Sequential, Model

In [ ]:
# X_mor_sequence_train = [str(x) for x in X_mor_sequence_train]
tokenizer = Tokenizer(lower=False)
tokenizer.fit_on_texts(X_mor_sequence_train)

X_train_seq = tokenizer.texts_to_sequences(X_mor_sequence_train)
X_val_seq = tokenizer.texts_to_sequences(X_mor_sequence_val.tolist())
X_test_seq = tokenizer.texts_to_sequences(X_mor_sequence_te)

### 4-2. 1-D CNN

### 4-3. LSTM

## 5. Using pre-trained model(Optional)
* 한국어 pre-trained model로 fine tuning 및 성능 분석
> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)
> * [HuggingFace-Korean](https://huggingface.co/models?language=korean)

In [ ]:
!pip install transformers
!pip install torch

In [ ]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('안녕하세요')
print(result)

loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.fc4c9d144e9e0cd8cfa0ba00f258c8f9355e6cfa7352a7650ef401642bc1a486
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.83

['안', '##녕', '##하', '##세', '##요']


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in X_tfidf_train]

loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.fc4c9d144e9e0cd8cfa0ba00f258c8f9355e6cfa7352a7650ef401642bc1a486
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.83

AttributeError: ignored

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader

# tokenizer와 pretrained model 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')

# dataset 불러오기
# train_dataset = CustomDataset(X_tfidf_train, y_train)
# eval_dataset = CustomDataset(X_tfidf_val, y_val)
train_dataset = (X_tfidf_train, y_train, X_tfidf_train.shape[0])
eval_dataset = (X_tfidf_val, y_val, X_tfidf_val.shape[0])

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps = 2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy='steps',
    eval_steps=500,
    load_best_model_at_end=True
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, max_length=512)
# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator = data_collator
)

# fine tuning 진행
trainer.train()

# 평가
trainer.evaluate()


loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.fc4c9d144e9e0cd8cfa0ba00f258c8f9355e6cfa7352a7650ef401642bc1a486
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/46880f3b0081fda494a4e15b05787692aa4c1e21e0ff2428ba8b14d4eda0784d.83

TypeError: ignored

In [ ]:
!pip install --upgrade kobert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.10.1-cp39-cp39-manylinux1_x86_64.whl (881.9 MB)
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.13.3
    Uninstalling huggingface-hub-0.13.3:
      Successfully uninstalled huggingface-hub-0.13.3
  Attempting uninstall: transformers
    Found existing ins

In [ ]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

In [ ]:
pip install torch==2.0.0 torchvision==0.15.1+cu118 torchtext==0.15.1 torchdata==0.6.0 torchaudio==2.0.1+cu118 -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-2.0.0-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.1
    Uninstalling torch-1.10.1:
      Successfully uninstalled torch-1.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobert 0.2.3 requires torch<=1.10.1,>=1.7.0, but you have torch 2.0.0 which is incompatible.


In [ ]:
pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-64wnlb9i
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-64wnlb9i
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00:

In [ ]:
pip install torch==1.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-1.10.1-cp39-cp39-manylinux1_x86_64.whl (881.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.10.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.10.1 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.10.1 which is incompatible.
torchaudio 2.0.1+cu118 requires torch==2.0.0, but you have torch 1.10.1 which is incompatible.
kobert 0.2.3 requires transformers<=4.8.1,>=4.8.1, but you have transformers 4.27.4 which is incompatible.


In [ ]:
!pip install transformers==4.8.1
!pip install kobert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.13.3
    Uninstalling huggingface-hub-0.13.3:
      Successfully uninstalled huggingface-hub-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.27.4
    Uninstalling transformers-4.27.4:
      Successfully uninstalled transformers-4.27.4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from kobert import get_pytorch_kobert_model

model, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('skt/kobert-base-v1')

AttributeError: ignored